This project aims to create a neural network capable of estimating DoA from telecommunications signals.

To do so, the training dataset consists of In-Phase and Quadrature (IQ) samples and Angle of Arrival (AoA) measures.
IQ samples consists

In [19]:
# Import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras import layers, models
import os
from keras.optimizers import Adam

In [7]:
# Set dataset path variable
dataset_path = os.path.join(
    os.getcwd(), "Matlab", "bluetooth_signals_dataset2024-02-05_22-56-11.csv"
)

In [8]:
# Load dataset
data = pd.read_csv(dataset_path)

In [46]:
# Extract features (IQ samples) and labels (Angles)
X_complex_str = data.iloc[:, 1:].values.astype(str)
y_str = data.iloc[:, 0].values

# Convert string representations of complex numbers to actual complex values for features
X_complex = np.array([np.complex128(complex(val.replace('i', 'j'))) for row in X_complex_str for val in row])
X_complex = X_complex.reshape(X_complex_str.shape)

# Separate real and imaginary parts for features
X_real = np.real(X_complex)
X_imag = np.imag(X_complex)

# Combine real and imaginary parts into a single array for features
X_combined = np.stack((X_real, X_imag), axis=-1)

# Convert string representations of complex numbers to actual complex values for labels
y_complex = np.array([np.complex128(complex(val.replace('i', 'j'))) for val in y_str])

# Use only the real part for labels
y = np.real(y_complex)

# Split the combined data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Standardize the input data
scaler = StandardScaler()
X_train_flat = X_train.reshape((-1, 16))
X_test_flat = X_test.reshape((-1, 16))
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)
X_train = X_train_scaled.reshape((-1, 4, 4, 2))
X_test = X_test_scaled.reshape((-1, 4, 4, 2))

# Model architecture
model = models.Sequential()
# Tentar mudar a arquitetura: uma camada inical maior, 2 camadas convolucionais, etc.
# Tentar um dataset com menos angulos, ao invés de 361 entre -90 e 90, de 5 em 5.
model.add(layers.Conv2D(16, (2, 2), activation='relu', input_shape=(4, 4, 2)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(36, activation='relu'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [47]:
"""
We'll use the `ModelCheckpoint` callback to regularly save checkpoints, and
the `EarlyStopping` callback to interrupt training when the validation loss
is not longer improving.
"""

path_checkpoint = "aoa_model_checkpoint.weights.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="mae", min_delta=0, patience=50, restore_best_weights=True)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="mae",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

In [48]:
# Train the model
model.fit(X_train, y_train, epochs=1000, validation_split=0.2)

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error on Test Set: {mae}")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/1000
8/8 [==============================] - 1s 27ms/step - loss: 10104.1562 - mae: 86.0546 - val_loss: 11437.0068 - val_mae: 93.3794
Epoch 2/1000
8/8 [==============================] - 0s 6ms/step - loss: 10103.4883 - mae: 86.0515 - val_loss: 11436.7607 - val_mae: 93.3808
Epoch 3/1000
8/8 [==============================] - 0s 6ms/step - loss: 10102.6738 - mae: 86.0478 - val_loss: 11436.4590 - val_mae: 93.3822
Epoch 4/1000
8/8 [==============================] - 0s 6ms/step - loss: 10101.7725 - mae: 86.0429 - val_loss: 11436.2607 - val_mae: 93.3830
Epoch 5/1000
8/8 [==============================] - 0s 6ms/step - loss: 10101.0361 - mae: 86.0398 - val_loss: 11435.5430 - val_mae: 93.3837
Epoch 6/1000
8/8 [==============================] - 0s 6ms/step - loss: 10099.7656 - mae: 86.0327 - val_loss: 11434.9961 - val_mae: 93.3835
Epoch 7/1000
8/8 [==============================] - 0s 6ms/step - loss: 10098.6377 - mae: 86.0277 - val_loss: 11434.6426 - val_mae: 93.3828
Epoch 8/1000
8/8 [=

In [77]:
predictions

array([[ 0.34495637],
       [-0.40295032],
       [ 0.17824353],
       ...,
       [ 0.48004445],
       [-0.29392853],
       [ 0.5742067 ]], dtype=float32)